# Data Collection

Data for prediction can either collected from Kaggle or Poloniex. To make sure coherence, the column names for data collected from Poloniex are changed to match with Kaggle’s.

In [13]:
import json
import numpy as np
import os
import pandas as pd
import urllib
from urllib.request import urlopen

# connect to poloniex's API
url_0 = 'https://poloniex.com/public?command=returnChartData&currencyPair=USDT_ETH&start=1612860000&end=1618242600&period=300'

url = [url_0]
length = [17943]
start = 1612860000
end = 1621499700

url_left = 'https://poloniex.com/public?command=returnChartData&currencyPair=USDT_ETH&start='
url_mid = '&end='
url_right = '&period=300'

for i in range(18):
    start -= 8640000 
    end -= 8640000
    url_cur = url_left + str(start) + url_mid + str(end) + url_right
    url.append(url_cur)
    length.append(28800)

length[17] = 28798
length[9] = 28799

# parse json returned from the API to Pandas DF
openUrl = urlopen(url.pop())
r = openUrl.read()
openUrl.close()
d = json.loads(r.decode())
df = pd.DataFrame(d, index=[i for i in range(length.pop())], columns=['date','high','low','open','close'])

original_columns=[u'date', u'high', u'low', u'open', u'close']
new_columns = ['Timestamp', 'High', 'Low', 'Open', 'Close']
df = df.loc[:,original_columns]
df.columns = new_columns
df.to_csv('data/ethereum2016to2021.csv',index=None)

for i in range(len(url)-1, -1, -1):
    openUrl = urlopen(url[i])
    r = openUrl.read()
    openUrl.close()
    d = json.loads(r.decode())
    df = pd.DataFrame(d, index=[i for i in range(length[i])], columns=['date','high','low','open','close'])

    df = df.loc[:,original_columns]
    df.columns = new_columns
    df.to_csv('data/ethereum2016to2021.csv', mode='a', index=None, header=False)

df.head()

,Timestamp,High,Low,Open,Close
0,1612860000,1802.909884,1791.473678,1799.742687,1796.696858
1,1612860300,1800.208498,1794.929996,1796.607035,1800.208498
2,1612860600,1808.000000,1801.000000,1801.000000,1805.780944
3,1612860900,1806.652680,1799.728182,1806.652680,1801.462288
4,1612861200,1808.826969,1802.280236,1802.280236,1804.344718


# Data Preparation

Data collected from source needs to be parsed in order to send to the model for prediction. The PastSampler class is for splitting the data into a list of datas and labels. The input size (N) is 256, while the output size (K) is 16. Note that data collected from Poloniex was ticked on a 5 minute basis. This indicates that, if the input spans across 1280 minutes, the output covers over 80 minutes

In [14]:
class PastSampler:
    '''
    Forms training samples for predicting future values from past value
    '''
     
    def __init__(self, N, K, sliding_window = True):
        '''
        Predict K future sample using N previous samples
        '''
        self.K = K
        self.N = N
        self.sliding_window = sliding_window
 
    def transform(self, A):
        M = self.N + self.K     #Number of samples per row (sample + target)
        #indexes
        if self.sliding_window:
            I = np.arange(M) + np.arange(A.shape[0] - M + 1).reshape(-1, 1)
        else:
            if A.shape[0]%M == 0:
                I = np.arange(M)+np.arange(0,A.shape[0],M).reshape(-1,1)
                
            else:
                I = np.arange(M)+np.arange(0,A.shape[0] -M,M).reshape(-1,1)
            
        B = A[I].reshape(-1, M * A.shape[1], A.shape[2])
        ci = self.N * A.shape[1]    #Number of features per sample
        return B[:, :ci], B[:, ci:] #Sample matrix, Target matrix

#data file path
dfp = 'data/ethereum2016to2021.csv'

#Columns of price data to use
columns = ['Close']
df = pd.read_csv(dfp)
time_stamps = df['Timestamp']
df = df.loc[:,columns]
original_df = pd.read_csv(dfp).loc[:,columns]

After creating the PastSampler class, we apply it on the collected data. Since the original data ranges from 0 to over 60000, data scaling is needed to allow the neural network to understand the data easier.

In [15]:
file_name='ethereum2016to2021_close.h5'

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# normalization
for c in columns:
    df[c] = scaler.fit_transform(df[c].values.reshape(-1,1))
    
#Features are input sample dimensions(channels)
A = np.array(df)[:,None,:]
original_A = np.array(original_df)[:,None,:]
time_stamps = np.array(time_stamps)[:,None,None]

#Make samples of temporal sequences of pricing data (channel)
NPS, NFS = 256, 16         #Number of past and future samples
ps = PastSampler(NPS, NFS, sliding_window=False)
B, Y = ps.transform(A)
input_times, output_times = ps.transform(time_stamps)
original_B, original_Y = ps.transform(original_A)

import h5py
with h5py.File(file_name, 'w') as f:
    f.create_dataset("inputs", data = B)
    f.create_dataset('outputs', data = Y)
    f.create_dataset("input_times", data = input_times)
    f.create_dataset('output_times', data = output_times)
    f.create_dataset("original_datas", data=np.array(original_df))
    f.create_dataset('original_inputs',data=original_B)
    f.create_dataset('original_outputs',data=original_Y)